In [2]:
!pip install tkinter

ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import tkinter as tk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import websocket
import json
import numpy as np

class CMEApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Aditya-L1 & SOHO CME Monitor")
        self.data = {
            'flux_proton': [], 'flux_sector': [], 'flux_alpha': [], 'threshold': [],
            'ahe': [], 'uncer': [], 'principal_angle': [], 'angular_width': [], 'velocity': [],
            'height_time_data': [], 'cme_detections': [], 'catalog_cmes': [], 'software_cmes': [],
            'is_cme_particle': False, 'is_cme_lasco': False, 'lasco_time': ''
        }

        # Create figures
        self.fig_heatmap, self.ax_heatmap = plt.subplots()
        self.fig_front, self.ax_front = plt.subplots()
        self.fig_map, self.ax_map = plt.subplots()

        # Initialize canvases
        self.canvas_heatmap = FigureCanvasTkAgg(self.fig_heatmap, master=root)
        self.canvas_front = FigureCanvasTkAgg(self.fig_front, master=root)
        self.canvas_map = FigureCanvasTkAgg(self.fig_map, master=root)

        # Pack canvases
        self.canvas_heatmap.get_tk_widget().pack()
        self.canvas_front.get_tk_widget().pack()
        self.canvas_map.get_tk_widget().pack()

        # Connect to WebSocket
        self.ws = websocket.WebSocketApp(
            "ws://localhost:5000",
            on_message=self.on_message,
            on_error=self.on_error,
            on_close=self.on_close
        )
        self.ws.on_open = self.on_open
        self.ws.run_forever()

    def on_open(self, ws):
        print("WebSocket Connected")

    def on_message(self, message):
        try:
            data = json.loads(message)
            self.update_data(data)
            self.update_plots()
        except Exception as e:
            print(f"Error parsing message: {e}")

    def on_error(self, ws, error):
        print(f"WebSocket Error: {error}")

    def on_close(self, ws, close_status_code, close_msg):
        print("WebSocket Closed")

    def update_data(self, new_data):
        self.data['flux_proton'].append(new_data['flux_proton'])
        self.data['flux_sector'].append(new_data['flux_sector'])
        self.data['flux_alpha'].append(new_data['flux_alpha'])
        self.data['threshold'].append(new_data['threshold'])
        self.data['ahe'].append(new_data['ahe'])
        self.data['uncer'].append(new_data['uncer'])
        self.data['principal_angle'].append(new_data['principal_angle'])
        self.data['angular_width'].append(new_data['angular_width'])
        self.data['velocity'].append(new_data['velocity'])
        self.data['height_time_data'] = new_data['height_time_data'] if new_data['height_time_data'] else self.data['height_time_data']
        self.data['cme_detections'] = new_data['cme_detections'] if new_data['cme_detections'] else self.data['cme_detections']
        self.data['catalog_cmes'] = new_data['catalog_cmes'] if new_data['catalog_cmes'] else self.data['catalog_cmes']
        self.data['software_cmes'] = new_data['software_cmes'] if new_data['software_cmes'] else self.data['software_cmes']
        self.data['is_cme_particle'] = new_data['is_cme_particle']
        self.data['is_cme_lasco'] = new_data['is_cme_lasco']
        self.data['lasco_time'] = new_data['lasco_time']

        # Limit to last 100 points
        for key in ['flux_proton', 'flux_sector', 'flux_alpha', 'threshold', 'ahe', 'uncer',
                   'principal_angle', 'angular_width', 'velocity']:
            self.data[key] = self.data[key][-100:]

    def update_plots(self):
        # [Height, Time] Plot
        self.ax_heatmap.clear()
        heatmap = self.ax_heatmap.imshow(self.data['height_time_data'], cmap='viridis', aspect='auto')
        self.ax_heatmap.set_xlabel('Time (minutes)')
        self.ax_heatmap.set_ylabel('Height (solar radii)')
        self.ax_heatmap.set_title('[Height, Time] Plot')
        for det in self.data['cme_detections']:
            t, angle, h = det
            x = (t - self.data['cme_detections'][0][0]) / (60 * 720) * heatmap.get_array().shape[1]
            y = (1 - (h - 2) / 28) * heatmap.get_array().shape[0]
            self.ax_heatmap.plot([x, x + 10], [y, y - 10], color='red', linewidth=2)
        self.canvas_heatmap.draw()

        # CME Front Tracking
        self.ax_front.clear()
        self.ax_front.plot(self.data['principal_angle'], label='Current Front', color='red')
        self.ax_front.plot([a - 5 for a in self.data['principal_angle']], label='Earlier', color='blue')
        self.ax_front.plot([a + 5 for a in self.data['principal_angle']], label='Later', color='yellow')
        self.ax_front.set_xlabel('Time Index')
        self.ax_front.set_ylabel('Principal Angle (°)')
        self.ax_front.set_title('CME Front Tracking')
        self.ax_front.legend()
        self.canvas_front.draw()

        # Combined Map
        self.ax_map.clear()
        catalog_x = [c[1] for c in self.data['catalog_cmes']]
        catalog_y = [(c[0] - self.data['catalog_cmes'][0][0]) / (60 * 720) for c in self.data['catalog_cmes']]
        software_x = [s[1] for s in self.data['software_cmes']]
        software_y = [(s[0] - self.data['software_cmes'][0][0]) / (60 * 720) for s in self.data['software_cmes']]
        self.ax_map.scatter(catalog_x, catalog_y, color='green', label='Catalog CMEs')
        self.ax_map.scatter(software_x, software_y, color='white', label='Software CMEs')
        self.ax_map.set_xlabel('Poloidal Angle (°)')
        self.ax_map.set_ylabel('Time (minutes)')
        self.ax_map.set_title('Combined CME Map')
        self.ax_map.legend()
        self.canvas_map.draw()

        # Alert for CME detection
        if self.data['is_cme_particle'] or self.data['is_cme_lasco']:
            print(f"Halo CME Detected at {self.data['lasco_time']}: Particle={self.data['is_cme_particle']}, LASCO={self.data['is_cme_lasco']}")

if __name__ == "__main__":
    root = tk.Tk()
    app = CMEApp(root)
    root.mainloop()

WebSocket Error: [WinError 10061] No connection could be made because the target machine actively refused it
WebSocket Closed


In [1]:
import numpy as np
import pandas as pd
import cdflib

cdf = cdflib.CDF('cdf_data/20250622/AL1_ASW91_L2_TH2_20250622_UNP_9999_999999_V02.cdf')

# Extract variables
epoch = cdflib.cdfepoch.to_datetime(cdf.varget('epoch_for_cdf_mod'))
energy = cdf.varget('energy_center_mod')
flux = cdf.varget('integrated_flux_mod')
xpos = cdf.varget('spacecraft_xpos')
ypos = cdf.varget('spacecraft_ypos')
zpos = cdf.varget('spacecraft_zpos')

# Assemble into a DataFrame using the first energy and flux channel
df = pd.DataFrame({
    'epoch': epoch,
    'energy': energy[:, 0],  # Use the first column
    'flux': flux[:, 0],      # Use the first column
    'x': xpos,
    'y': ypos,
    'z': zpos
})


In [2]:
df

,epoch,energy,flux,x,y,z
0,2025-06-22 00:00:28.027,172.501303,NaN,1.274104e+06,-27485.198361,-106418.360875
1,2025-06-22 00:00:33.027,172.501303,NaN,1.274104e+06,-27483.697389,-106418.365237
2,2025-06-22 00:00:38.027,172.501303,442708.333333,1.274104e+06,-27482.196446,-106418.369598
3,2025-06-22 00:00:43.027,172.501303,NaN,1.274104e+06,-27480.695503,-106418.373958
4,2025-06-22 00:00:48.027,172.501303,NaN,1.274104e+06,-27479.194529,-106418.378318
...,...,...,...,...,...,...
17270,2025-06-22 23:59:38.806,172.501303,797916.666667,1.274817e+06,-1471.881323,-106410.345761
17271,2025-06-22 23:59:43.806,172.501303,NaN,1.274817e+06,-1470.370550,-106410.340494
17272,2025-06-22 23:59:48.806,172.501303,NaN,1.274817e+06,-1468.859777,-106410.335226
17273,2025-06-22 23:59:53.806,172.501303,NaN,1.274817e+06,-1467.348974,-106410.329958
